In [17]:

# this is the gpl method simplified from:
#https://github.com/pinecone-io/examples/blob/master/analytics-and-ml/model-training/gpl/README.md
# with small changes and applied to Finreg-E by I. Achitouv 2023 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from spacy.lang.en import English
import re

from numpy.linalg import norm 

import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize,word_tokenize
import torch
from transformers import AutoTokenizer, AutoModel,AutoModelForMaskedLM

from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments



#Cleaning functions
def sentence_clean(text):

     # removing mentions
    text = re.sub("@\S+", "", text)
     # remove urls
    text = re.sub("https?:\/\/.*[\r\n]*", "", text)
     # removing hashtags
    text = re.sub("#", "", text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    # remove digits
    text = re.sub("\d+", "", text)
    # remove bracket
    text=re.sub(r"[\([{})\]]", "", text)
    text=re.sub(r"\([^()]*\)", "", text)

    return text


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"


def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")

    text = text.replace(":"," ")
    text = text.replace(","," ")
    text = text.replace('.'," ")
    text = text.replace(']'," ")
    text = text.replace('['," ")
    text = text.replace(','," ")
    text = text.replace("'"," ")
    text = text.replace('('," ")
    text = text.replace('’'," ")
    text = text.replace(')'," ")
    text = text.replace('“'," ")
    text = text.replace('”'," ")
    text = text.replace(';'," ")
    text = text.replace(','," ")
    text = text.replace('//'," ")
    text = text.replace('``'," ")
    text = text.replace("''"," ")
    text = text.replace("'-"," ")
    text = text.replace('\\\\'," ")
    # remove digits
    text = re.sub("\d+", "", text)
    # remove bracket
    text=re.sub(r"[\([{})\]]", "", text)

    text=re.sub(r"\([^()]*\)", "", text)
   # text = text.replace(," ")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences


def cleancorpus(Corpus):
    Corpus_s=[]
    for i in range(len(Corpus)):
        t=split_into_sentences(Corpus[i])
        for j in range(len(t)):
            t2=t[j]
            if len(t2)>2:
                Corpus_s.append(t2)


    cleanCorpus=[]
    for i in range(len(Corpus_s)):
        cleanCorpus.append(sentence_clean(Corpus_s[i]))

    #datas=random.shuffle(cleanCorpus)
    return cleanCorpus


def splitchunk(data,lchunck):
    data2=[]
    for sent in data:
        if len(sent)<lchunck:
            data2.append(sent)
        else:
            for i in range(0,len(sent),lchunck):
                data2.append(sent[i:i+lchunck])
    return data2




import random


from tqdm.auto import tqdm
import json
import os
import sqlite3


#### read data corpus

db_file ="/Users/ixandra/MEGA/Sync/NLP/Finreg/Moody/onhpc/Rulebooks.db"
con = sqlite3.connect(db_file)
cur = con.cursor()


sqlQry = "select * from FCARulebook_FEB22"
dataList = pd.read_sql_query(sqlQry, con)
    # Be sure to close the connection
con.close()


print(len(dataList))
TextRul=[]
ic=0
for i in range(0, len(dataList)):
    dataToPrint = dataList['RegulatorRuleFullRequirement'][i]
    if dataList['RegulatorRuleFullRequirement'][i]!="[deleted]":
        TextRul.append(dataToPrint)
        ic+=1
print("there was",ic,"rules",len(TextRul))

data=cleancorpus(TextRul)
random.shuffle(data)
#train_sentences=splitchunk(data,200)
passages=data
print(len(data),"len data")
passages=TextRul

21915
there was 20306 rules 20306
73254 len data


In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'doc2query/msmarco-t5-base-v1'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)#.cuda()

In [212]:
import os

if not os.path.exists('data'): os.mkdir('data')


In [14]:
from tqdm.auto import tqdm  # this is our progress bar

target = 15000
batch_size = 300
num_queries = 2  # number of queries to generate for each passage
count = 0
lines = []
passage_batch = []

# reinitialize passage generator
#passages = get_text()

with tqdm(total=target) as progress:
    for passage in passages:
        if count >= target: break
        # remove tab + newline characters if present
        passage_batch.append(passage.replace('\t', ' ').replace('\n', ' '))
        
        # we encode in batches
        if len(passage_batch) == batch_size:
            # tokenize the passage
            inputs = tokenizer(
                passage_batch,
                truncation=True,
                padding=True,
                max_length=256,
                return_tensors='pt'
            )

            # generate three queries per doc/passage
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=64,
                do_sample=True,
                top_p=0.95,
                num_return_sequences=num_queries
            )

            # decode query to human readable text
            decoded_output = tokenizer.batch_decode(
                outputs,
                skip_special_tokens=True
            )

            # loop through to pair query and passages
            for i, query in enumerate(decoded_output):
                query = query.replace('\t', ' ').replace('\n', ' ')  # remove newline + tabs
                passage_idx = int(i/num_queries)  # get index of passage to match query
                lines.append(query+'\t'+passage_batch[passage_idx])
                count += 1
            
            passage_batch = []
            progress.update(len(decoded_output))

# write (Q, P+) pairs to file
with open('./data/pairs.tsv', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(lines))


  0%|          | 0/15000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [214]:
############ PART 2 independant of PART 1

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256
model


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [215]:
# install pinecone on your pc and register for free to get api key 
import pinecone  # pip install pinecone-client

API_KEY = "put your pinecode key"  # get api key app.pinecone.io

pinecone.init(
    api_key=API_KEY,
    environment="put your environment name"  # find next to API key in console
)
# create a new negative mining index if does not already exist
if 'negative-mine' not in pinecone.list_indexes():
    pinecone.create_index(
        'negative-mine',
        dimension=model.get_sentence_embedding_dimension(),
        metric='dotproduct',
        pods=1
    )
# connect
index = pinecone.Index('negative-mine')


In [ ]:
#Now we encode the passages and store in the negative-mine index.

In [216]:
from tqdm.auto import tqdm

def get_text():
    with open('data/pairs.tsv', 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    for line in tqdm(lines):
        try:
            query, passage = line.split('\t')
            yield query, passage
        except ValueError:
            # in case of malformed data, pass onto next row
            pass

In [217]:
pair_gen = get_text()

pairs = []
to_upsert = []
passage_batch = []
id_batch = []
batch_size = 64

for i, (query, passage) in enumerate(pair_gen):
    pairs.append((query, passage))
    # we do this to avoid passage duplication in the vector DB
    if passage not in passage_batch: 
        passage_batch.append(passage)
        id_batch.append(str(i))
    # on reaching batch_size, we encode and upsert
    if len(passage_batch) == batch_size:
        embeds = model.encode(passage_batch).tolist()
        # upload to index
        index.upsert(vectors=list(zip(id_batch, embeds)))
        # refresh batches
        passage_batch = []
        id_batch = []
        
# check number of vectors in the index
index.describe_index_stats()

  0%|          | 0/20736 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6978}},
 'total_vector_count': 6978}

In [218]:
# I edited this part to make it work on my mac 
import random

batch_size = 1
triplets = []

for i in tqdm(range(0, len(pairs))):
    # embed queries and query pinecone in batches to minimize network latency
    #i_end = min(i+batch_size, len(pairs))
    queries = pairs[i][0] #[pair[0] for pair in pairs[i:i_end]]
    pos_passages =pairs[i][1] #[pair[1] for pair in pairs[i:i_end]]
    
    # create query embeddings
    query_embs = model.encode(queries, convert_to_tensor=True, show_progress_bar=False)
    # search for top_k most similar passages
    res = index.query(query_embs.tolist(), top_k=10)
    # iterate through queries and find negatives
    #for query, pos_passage, query_res in zip(queries, pos_passages, res):
    pos_passage=str(pos_passages)
    
    top_results = res['matches']
    # shuffle results so they are in random order
    random.shuffle(top_results)
    for hit in top_results:
        neg_passage = pairs[int(hit['id'])][1]
            # check that we're not just returning the positive passage
            
        if neg_passage != pos_passage:
                # if not we can add this to our (Q, P+, P-) triplets
            triplets.append(queries+'\t'+pos_passage+'\t'+neg_passage)
            break

with open('data/triplets.tsv', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(triplets))


  0%|          | 0/20736 [00:00<?, ?it/s]

In [199]:
pinecone.delete_index('negative-mine')  # delete the index when done to avoid higher charges (if using multiple pods)


NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Thu, 16 Feb 2023 21:09:50 GMT', 'x-envoy-upstream-service-time': '4', 'content-length': '14', 'server': 'envoy'})
HTTP response body: 404: Not Found


In [ ]:
######### start independant notebook 3

In [219]:
#With that we now have even more (query, passage) pairs, that are both positive and negative matches. 
#The next step in GPL will see us scoring all of these pairs
#using a cross-encoder model.

#We pass these into a cross encoder model that is trained to predict
#similarity scores of (Q, P) pairs.
#First, we will load our cross encoder model.

from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
model

In [220]:
#Let's define a generator function to read pairs from file.

from tqdm.auto import tqdm

def get_lines():
    # loop through each file
    with open('data/triplets.tsv', 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    # loop through each line in the current file
    for line in tqdm(lines):
        q, p, n = line.split('\t')
        # return the query, positive, negative
        yield q, p, n
#We use the cross encoder to calculate the similarity of the positive
#pair, and negative pair, and then take the score as the margin between the two similarity scores. We are taking the margin as we will be training our bi-encoder model using Margin MSE loss (which requires the margin/difference).
lines = get_lines()
label_lines = []

for line in lines:
    q, p, n = line
    p_score = model.predict((q, p))
    n_score = model.predict((q, n))
    margin = p_score - n_score
    # append pairs to label_lines with margin score
    label_lines.append(
        q + '\t' + p + '\t' + n + '\t' + str(margin)
    )

with open("data/triplets_margin.tsv", 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(label_lines))
    
#Now we have our (Q, P+, P-) pairs, we can move on to fine-tuning with MarginMSE loss.
    

  0%|          | 0/20683 [00:00<?, ?it/s]

In [221]:
#Fine-tuning with MSEMargin Loss
#Now that we have our margin labeled (Q, P+, P-) pairs,
#we can begin fine-tuning a bi-encoder model with MSEMargin loss. We will start by defining a data loading function that uses the standard InputExample format of sentence-transformers.
from tqdm.auto import tqdm
from sentence_transformers import InputExample

training_data = []

with open('data/triplets_margin.tsv', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')
# loop through each line and return InputExample
for line in tqdm(lines):
    q, p, n, margin = line.split('\t')
    training_data.append(InputExample(
        texts=[q, p, n],
        label=float(margin)
    ))

len(training_data)



#We load these pairs into a generator DataLoader. Margin MSE works best with a large batch_size,
#the 32 used here is reasonable.

import torch

torch.cuda.empty_cache()

batch_size = 32

loader = torch.utils.data.DataLoader(
    training_data, batch_size=batch_size, shuffle=True
)


#Next we initialize a bi-encoder model that we will be 
#fine-tuning using domain adaption
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256
model

#Then initialize the Margin MSE loss function.
from sentence_transformers import losses

loss = losses.MarginMSELoss(model)

  0%|          | 0/20683 [00:00<?, ?it/s]

In [222]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='msmarco-distilbert-base-FinregE',
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/647 [00:00<?, ?it/s]